### Ageing_Sign_Detect

In [ ]:
from google.colab.patches import cv2_imshow
from keras.models import model_from_json # loading the model from json
from collections import OrderedDict
from imutils import face_utils
import imutils
import cv2
import dlib
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
#loading the model and weights
#################PATH TO model.json FILE COMES HERE##################
json_file = open("path_to_model.json_file", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
##########################PATH TO weights.h5 FILE COMES HERE##################
loaded_model.load_weights("path_to_weights.h5_file")
print("Model Loaded Successfully!!")

In [ ]:


def visualize_facial_landmarks(image, shape, colors=None, alpha=0.6):

  FACIAL_LANDMARKS_IDXS = OrderedDict([("left_eye", (42, 48)),
                                       ("right_eye", (36, 42))])

  overlay = image.copy()
  output = image.copy()

  if colors is None:
    colors = (19, 199, 109)

  for  (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):

    (j, k) = FACIAL_LANDMARKS_IDXS[name]
    pts = shape[j:k]
    
    for index, element in enumerate(pts):
      pts[index] = [element[0], element[1]+8]

    hull = cv2.convexHull(pts)
    cv2.drawContours(overlay, [hull], -1, colors, -1)
    
  cv2.addWeighted(overlay, alpha, output, 1-alpha, 0, output)

  return output


In [ ]:


def puffy_localise(image):
    
    detector = dlib.get_frontal_face_detector()
    #################################PATH TO shape-predictor.dat FILE COMES HERE####################################
    predictor = dlib.shape_predictor("path_to_shape-predictor.dat")
    
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)

    for (i, rect) in enumerate(rects):
      shape = predictor(gray, rect)
      shape = face_utils.shape_to_np(shape)

    output = visualize_facial_landmarks(image, shape)
    return output

In [ ]:
# Testing the model against random images taken from the internet
###################PATH TO image FILE COMES HERE##############
image = cv2.imread("path_to_image_file")
image = imutils.resize(image, width=500)
#####################################PATH TO cascade.xml FILE COMES HERE#########################################
face_cascade = cv2.CascadeClassifier("path_to_cascade.xml_file")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.05, 5)

In [ ]:
labels = np.array(['dark', 'notOld', 'puffy', 'wrinkles'], dtype=object)

for (x, y, w, h) in faces:
  detectedFace = image[x:x+w, y:y+h]
  detectedFace = cv2.resize(detectedFace, (244, 244))
  detectedFace = detectedFace.reshape(1, 244, 244, 3)
  predictions=loaded_model.predict(detectedFace)[0]
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
  
  count = 1
  for i in predictions:
    classname = labels[np.where(predictions == i)]
    accuracy = i*100
    label = "{}: {:.2f}%".format(classname, accuracy)
    
    if classname == 'puffy' and accuracy > 75:
      image = puffy_localise(image)
    
    cv2.putText(image, label, (10, count*30+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    count += 1
  print(predictions)

  cv2_imshow(image)